# Статистическая и ML-обработка результатов голосования Госдумы (2016-2021)

### Парсер результатов голосования с сайта http://vote.duma.gov.ru/ . Были собраны все данные о голосованиях госдумы 7-го созыва (2016-2021 г.)

In [1]:
import requests
import numpy as np
import pandas as pd
import time
import json
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Диапазон номеров голосований
page_list = list(range(113991, 96006, -1))

In [3]:
len(page_list)

17985

In [31]:
unparsed = []
# Словарь с результатами голосований
votes = {}
# Словарь с партиной принадлежностью депутатов
dep_part_dict = {}
for page in tqdm(page_list):
    page_link = "http://vote.duma.gov.ru/vote/" + str(page)
    try:
        response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
        temp_str = str(soup.findAll('script')[10])
        str_pos = temp_str.find('deputiesData ')
        temp_str2 = temp_str[str_pos + 15 : temp_str.find(']', str_pos)+1]
        dip_list = json.loads(temp_str2)
        dep_dict = {}
        for d in dip_list:
            dep_dict[d['sortName']] = d['result']
            if d['sortName'] not in dep_part_dict:
                dep_part_dict[d['sortName']] = d['faction']

        law_name = soup.find('div', attrs = {'class':'box-p'}).h1.text
        vote_result = soup.find('div', attrs = {'class':'statis-p'}).b.text.split()[0]
        try:
            votes_yes = soup.find('div', attrs = {'class':'statis-p'}).b.text.split()[1][1:]
            t = soup.find('div', attrs = {'class':'statis-p'}).b.text.split()[3]
            votes_yes_perc = t[:t.find('%')]
        except:
            votes_yes = None
            votes_yes_perc = None
        votes[page] = [law_name, vote_result, votes_yes, votes_yes_perc, dep_dict]
    except:
        print(f"error, unparsed = {page}")
        unparsed.append(page)
        continue
    
    time.sleep(1 + np.random.rand())

error, unparsed = 99906
error, unparsed = 99905
error, unparsed = 99904
error, unparsed = 99903
error, unparsed = 99902
error, unparsed = 99901
error, unparsed = 99900
error, unparsed = 99899
error, unparsed = 99898
error, unparsed = 99897
error, unparsed = 99896
error, unparsed = 99895
error, unparsed = 99894
error, unparsed = 99893
error, unparsed = 99892
error, unparsed = 99891
error, unparsed = 99890
error, unparsed = 99889
error, unparsed = 99888
error, unparsed = 99887
error, unparsed = 99886
error, unparsed = 99885
error, unparsed = 99884
error, unparsed = 99883
error, unparsed = 99882
error, unparsed = 99881
error, unparsed = 99880
error, unparsed = 99879
error, unparsed = 99878
error, unparsed = 99877
error, unparsed = 99876
error, unparsed = 99875
error, unparsed = 99874
error, unparsed = 99873
error, unparsed = 99872
error, unparsed = 99871
error, unparsed = 99870
error, unparsed = 99869
error, unparsed = 99868
error, unparsed = 99867
error, unparsed = 99866
error, unparsed 

error, unparsed = 99299
error, unparsed = 99298
error, unparsed = 99297
error, unparsed = 99296
error, unparsed = 99295
error, unparsed = 99294
error, unparsed = 99293
error, unparsed = 99292
error, unparsed = 99291
error, unparsed = 99290
error, unparsed = 99289
error, unparsed = 99288
error, unparsed = 99287
error, unparsed = 99286
error, unparsed = 99285
error, unparsed = 99284
error, unparsed = 99283
error, unparsed = 99282
error, unparsed = 99281
error, unparsed = 99280
error, unparsed = 99279
error, unparsed = 99278
error, unparsed = 99277
error, unparsed = 99276
error, unparsed = 99275
error, unparsed = 99274
error, unparsed = 99273
error, unparsed = 99272
error, unparsed = 99271
error, unparsed = 99270
error, unparsed = 99269
error, unparsed = 99268
error, unparsed = 99267
error, unparsed = 99266
error, unparsed = 99265
error, unparsed = 99264
error, unparsed = 99263
error, unparsed = 99262
error, unparsed = 99261
error, unparsed = 99260
error, unparsed = 99259
error, unparsed 

error, unparsed = 98287
error, unparsed = 98286
error, unparsed = 98285
error, unparsed = 98284
error, unparsed = 98283
error, unparsed = 98282
error, unparsed = 98281
error, unparsed = 98280
error, unparsed = 98279
error, unparsed = 98278
error, unparsed = 98277
error, unparsed = 98276
error, unparsed = 98275
error, unparsed = 98274
error, unparsed = 98273
error, unparsed = 98272
error, unparsed = 98271
error, unparsed = 98270
error, unparsed = 98269
error, unparsed = 98268
error, unparsed = 98267
error, unparsed = 98224
error, unparsed = 98223
error, unparsed = 98222
error, unparsed = 98221
error, unparsed = 98220
error, unparsed = 98219
error, unparsed = 98218
error, unparsed = 98217
error, unparsed = 98216
error, unparsed = 98215
error, unparsed = 98214
error, unparsed = 98213
error, unparsed = 98212
error, unparsed = 98211
error, unparsed = 98210
error, unparsed = 98209
error, unparsed = 98208
error, unparsed = 98207
error, unparsed = 98206
error, unparsed = 98205
error, unparsed 

error, unparsed = 97522
error, unparsed = 97521
error, unparsed = 97520
error, unparsed = 97519
error, unparsed = 97518
error, unparsed = 97517
error, unparsed = 97516
error, unparsed = 97515
error, unparsed = 97514
error, unparsed = 97513
error, unparsed = 97512
error, unparsed = 97511
error, unparsed = 97510
error, unparsed = 97509
error, unparsed = 97508
error, unparsed = 97507
error, unparsed = 97506
error, unparsed = 97505
error, unparsed = 97504
error, unparsed = 97503
error, unparsed = 97502
error, unparsed = 97501
error, unparsed = 97500
error, unparsed = 97499
error, unparsed = 97498
error, unparsed = 97497
error, unparsed = 97496
error, unparsed = 97495
error, unparsed = 97494
error, unparsed = 97493
error, unparsed = 97492
error, unparsed = 97491
error, unparsed = 97490
error, unparsed = 97489
error, unparsed = 97488
error, unparsed = 97487
error, unparsed = 97486
error, unparsed = 97485
error, unparsed = 97484
error, unparsed = 97483
error, unparsed = 97482
error, unparsed 

In [5]:
a_file = open("./data/dep_part_dict.json", "w")
json.dump(dep_part_dict, a_file)
a_file.close()

In [35]:
# Построение таблицы с результатами голосований
dep_votes = {}
for v in votes:
    for d in votes[v][4]:
        if d in dep_votes:
            dep_votes[d][v] = votes[v][4][d]
        else:
            dep_votes[d] = {v: votes[v][4][d]}

In [36]:
len(dep_votes)

498

Сборка датафрейма с результатами голосований

In [44]:
df = pd.DataFrame(index=[v for v in votes])
df.head()

""
113991
113990
113989
113988
113987


In [42]:
df1 = pd.DataFrame([votes[v][0] for v in votes], index=[v for v in votes], columns=['law_name'])
df2 = pd.DataFrame([votes[v][1] for v in votes], index=[v for v in votes], columns=['vote_result'])

In [45]:
for d in dep_votes:
    df = df.join(pd.DataFrame([dep_votes[d][v] for v in dep_votes[d]], index=[v for v in dep_votes[d]], columns=[d]))

In [46]:
df.head()

,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Алимова Ольга Николаевна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,Андрейченко Андрей Валерьевич,Аникеев Григорий Викторович,...,Решульский Сергей Николаевич,Грищенко Олег Васильевич,Жутенков Владимир Александрович,Сафин Марат Мубинович,Тарасюк Василий Михайлович,Любимов Николай Викторович,Меткин Александр Михайлович,Скоробогатько Александр Иванович,Макаровец Николай Александрович,Нарышкин Сергей Евгеньевич
113991,-1,-1,-1,-1,-1,-1.0,-1,-1,-1.0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113990,2,2,2,2,2,2.0,2,2,2.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113989,2,2,2,-1,2,NaN,2,2,2.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113988,2,2,2,2,2,NaN,2,2,2.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113987,-1,2,-1,-1,2,NaN,-1,-1,-1.0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df_full = df1.join(df2).join(df)

In [48]:
df_full.head()

,law_name,vote_result,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Алимова Ольга Николаевна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,...,Решульский Сергей Николаевич,Грищенко Олег Васильевич,Жутенков Владимир Александрович,Сафин Марат Мубинович,Тарасюк Василий Михайлович,Любимов Николай Викторович,Меткин Александр Михайлович,Скоробогатько Александр Иванович,Макаровец Николай Александрович,Нарышкин Сергей Евгеньевич
113991,(за основу) О проекте порядка работы Государст...,принят,-1,-1,-1,-1,-1,-1.0,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113990,Регистрация 12:01.02,принят,2,2,2,2,2,2.0,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113989,(первое чтение) О проекте федерального закона ...,отклонен,2,2,2,-1,2,NaN,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113988,(первое чтение) О проекте федерального закона ...,отклонен,2,2,2,2,2,NaN,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113987,(первое чтение) О проекте федерального закона ...,принят,-1,2,-1,-1,2,NaN,-1,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
#df_full.to_csv('./data/df_full.csv')

In [22]:
#df_full = pd.read_csv('./data/df_full.csv', index_col=0)

In [28]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15544 entries, 113991 to 96007
Columns: 500 entries, law_name to Нарышкин Сергей Евгеньевич
dtypes: float64(103), int64(395), object(2)
memory usage: 59.4+ MB


In [70]:
df_full_cr = df_full.copy()

In [71]:
# Доля пропущенных голосований
df_full_cr.isnull().sum() / 15544

law_name                            0.000193
vote_result                         0.000000
Авдеев Александр Александрович      0.000000
Адучиев Батор Канурович             0.000000
Азимов Рахим Азизбоевич             0.000000
                                      ...   
Любимов Николай Викторович          0.906330
Меткин Александр Михайлович         0.906330
Скоробогатько Александр Иванович    0.965839
Макаровец Николай Александрович     0.992859
Нарышкин Сергей Евгеньевич          0.992859
Length: 500, dtype: float64

In [72]:
# Депутаты, которые пропустили более 10% голосований
t = (df_full_cr.isnull().sum() / 15544) > 0.1
t

law_name                            False
vote_result                         False
Авдеев Александр Александрович      False
Адучиев Батор Канурович             False
Азимов Рахим Азизбоевич             False
                                    ...  
Любимов Николай Викторович           True
Меткин Александр Михайлович          True
Скоробогатько Александр Иванович     True
Макаровец Николай Александрович      True
Нарышкин Сергей Евгеньевич           True
Length: 500, dtype: bool

In [73]:
sum(t[t == True])

91

In [74]:
# Удаляем депутатов с большим количеством пропусков (т.е. которые не были в составе думы в течение всего созыва)
df_full_cr = df_full_cr.drop(columns=(t[t == True].index))

In [75]:
# Для остальных заполняем пропуски значением 2 (не голосовал)
df_full_cr = df_full_cr.fillna(2)

In [76]:
df_full_cr.head(3)

,law_name,vote_result,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,Аникеев Григорий Викторович,...,Эмиргамзаев Абдулгамид Гасанович,Юмашева Инга Альбертовна,Юрков Дмитрий Васильевич,Ющенко Александр Андреевич,Ямпольская Елена Александровна,Яровая Ирина Анатольевна,Антошкин Николай Тимофеевич,Зиннуров Ирек Хайдарович,Кобзев Юрий Викторович,Шурчанов Валентин Сергеевич
113991,(за основу) О проекте порядка работы Государст...,принят,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,2.0,2.0,2.0,2.0
113990,Регистрация 12:01.02,принят,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2.0,2.0,2.0,2.0
113989,(первое чтение) О проекте федерального закона ...,отклонен,2,2,2,-1,2,2,2,2,...,2,2,2,2,2,2,2.0,2.0,2.0,2.0


In [83]:
df_full_cr_0 = df_full_cr.copy()

# Преобразование типов в int8
for d in df_full_cr.drop(columns=['law_name', 'vote_result']).columns:
    df_full_cr = df_full_cr.astype({d: 'int8'})
    df_full_cr_0 = df_full_cr_0.astype({d: 'int8'})
    
    # Приравниваем неголосовавших депутатов к воздержавшимся
    df_full_cr_0[d] = df_full_cr_0[d].map(lambda x: 0 if x == 2 else x)

In [84]:
df_full_cr.head()

,law_name,vote_result,Авдеев Александр Александрович,Адучиев Батор Канурович,Азимов Рахим Азизбоевич,Аксаков Анатолий Геннадьевич,Алексеева Татьяна Олеговна,Альшевских Андрей Геннадьевич,Ананских Игорь Александрович,Аникеев Григорий Викторович,...,Эмиргамзаев Абдулгамид Гасанович,Юмашева Инга Альбертовна,Юрков Дмитрий Васильевич,Ющенко Александр Андреевич,Ямпольская Елена Александровна,Яровая Ирина Анатольевна,Антошкин Николай Тимофеевич,Зиннуров Ирек Хайдарович,Кобзев Юрий Викторович,Шурчанов Валентин Сергеевич
113991,(за основу) О проекте порядка работы Государст...,принят,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,2,2,2,2
113990,Регистрация 12:01.02,принят,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
113989,(первое чтение) О проекте федерального закона ...,отклонен,2,2,2,-1,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
113988,(первое чтение) О проекте федерального закона ...,отклонен,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,-1
113987,(первое чтение) О проекте федерального закона ...,принят,-1,2,-1,-1,2,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [85]:
df_full_cr.shape

(15544, 409)

In [86]:
df_full_cr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15544 entries, 113991 to 96007
Columns: 409 entries, law_name to Шурчанов Валентин Сергеевич
dtypes: int8(407), object(2)
memory usage: 7.0+ MB


In [87]:
df_full_cr.to_csv('./data/df_full_cr.csv')
df_full_cr_0.to_csv('./data/df_full_cr_0.csv')